In [70]:
import open3d as o3d

mesh = o3d.io.read_triangle_mesh("../84degrees/Segmentation_femur_84deg.stl")
pointcloud = mesh.sample_points_poisson_disk(20000)

# you can plot and check
o3d.visualization.draw_geometries([mesh])
o3d.visualization.draw_geometries([pointcloud])

In [71]:
o3d.io.write_point_cloud("../femur_pcls/84deg.ply",pointcloud)

True

In [72]:
pcd_load = o3d.io.read_point_cloud("../femur_pcls/84deg.ply")
o3d.visualization.draw_geometries([pcd_load])

In [73]:
import numpy as np

In [74]:
pc_array = np.asarray(pcd_load.points)

In [75]:
pc_array.shape

(20000, 3)

In [76]:
print("Testing kdtree in Open3D...")
print("Load a point cloud and paint it blue.")
pcd_load.paint_uniform_color([0.2, 0.6, 0.8])
pcd_tree = o3d.geometry.KDTreeFlann(pcd_load)

Testing kdtree in Open3D...
Load a point cloud and paint it blue.


In [77]:
import pandas as pd

def in_cycle(center_x, center_y, radius, x, y):
    return (x - center_x)**2 + (y - center_y)**2 <= radius**2

In [78]:
df = pd.DataFrame(pc_array, columns=["x","y", "z"])

In [79]:
radius = 16
deg= 84
center_x = np.median(df['x'])
center_y = np.median(df['y'])

df_in_radius = df[df.apply(lambda x: in_cycle(center_x,center_y,radius, x.x,x.y), axis=1)]
local_minimum = df_in_radius.loc[df_in_radius.z.idxmin()]

In [80]:
local_minimum

x   -105.127490
y    118.499741
z     -9.111352
Name: 4500, dtype: float64

In [81]:
local_minimum=local_minimum.to_numpy()

In [82]:
print("Find its neighbors with distance less than 0.2, and paint them pink.")
[k, idx, temp1] = pcd_tree.search_radius_vector_3d(pcd_load.points[np.where(np.all(np.asarray(pcd_load.points)==local_minimum, axis=(1)))[0][0]], 4)
np.asarray(pcd_load.colors)[idx[1:], :] = [0.9, 0.6, 0.8]
np.asarray(pcd_load.colors)[np.where(np.all(np.asarray(pcd_load.points)==local_minimum, axis=(1)))[0][0]] = [1, 0, 0]

Find its neighbors with distance less than 0.2, and paint them pink.


In [83]:
print("Visualize the point cloud.")
o3d.visualization.draw_geometries([pcd_load],
                                  zoom=0.5599,
                                  front=[-0.4958, 0.8229, 0.2773],
                                  lookat=[2.1126, 1.0163, -1.8543],
                                  up=[0.1007, -0.2626, 0.9596])

Visualize the point cloud.


In [84]:
np.save("../femur_pcls/minimas/84deg.npy",local_minimum)

In [18]:
# radius_all=[]
# deg_all=[]

In [85]:
radius_all.append(radius)
deg_all.append(deg)

In [86]:
radius_all

[2, 7, 10, 16]

In [87]:
deg_all

[0, 23, 52, 84]

In [88]:
from sklearn.linear_model import LinearRegression

In [89]:
reg = LinearRegression().fit(np.reshape(deg_all,(-1,1)), np.reshape(radius_all,(-1,1)))

In [90]:
predicted_radius=reg.predict(np.array([[65]]))
print("Predicted radius=",predicted_radius[0][0])

Predicted radius= 12.782047244094485


## Testing Prediction

In [91]:
import open3d as o3d

mesh = o3d.io.read_triangle_mesh("../65degrees/Segmentation_femur_65deg.stl")
pointcloud = mesh.sample_points_poisson_disk(20000)

# you can plot and check
o3d.visualization.draw_geometries([mesh])
o3d.visualization.draw_geometries([pointcloud])

In [92]:
o3d.io.write_point_cloud("../femur_pcls/65deg.ply",pointcloud)

True

In [100]:
pcd_load = o3d.io.read_point_cloud("../femur_pcls/65deg.ply")
o3d.visualization.draw_geometries([pcd_load])

In [101]:
pc_array = np.asarray(pcd_load.points)

In [102]:
print("Testing kdtree in Open3D...")
print("Load a point cloud and paint it blue.")
pcd_load.paint_uniform_color([0.2, 0.6, 0.8])
pcd_tree = o3d.geometry.KDTreeFlann(pcd_load)

Testing kdtree in Open3D...
Load a point cloud and paint it blue.


In [103]:
df = pd.DataFrame(pc_array, columns=["x","y", "z"])

In [104]:
radius = predicted_radius[0][0]
deg=65
center_x = np.median(df['x'])
center_y = np.median(df['y'])

df_in_radius = df[df.apply(lambda x: in_cycle(center_x,center_y,radius, x.x,x.y), axis=1)]
local_minimum = df_in_radius.loc[df_in_radius.z.idxmin()]

In [105]:
local_maximum

array([-119.80013113,  102.3511234 ,   84.25270109])

In [106]:
local_minimum=local_minimum.to_numpy()

In [107]:
print("Find its neighbors with distance less than 0.2, and paint them pink.")
[k, idx, temp1] = pcd_tree.search_radius_vector_3d(pcd_load.points[np.where(np.all(np.asarray(pcd_load.points)==local_minimum, axis=(1)))[0][0]], 4)
np.asarray(pcd_load.colors)[idx[1:], :] = [0.9, 0.6, 0.8]
np.asarray(pcd_load.colors)[np.where(np.all(np.asarray(pcd_load.points)==local_minimum, axis=(1)))[0][0]] = [1, 0, 0]

Find its neighbors with distance less than 0.2, and paint them pink.


In [108]:
print("Visualize the point cloud.")
o3d.visualization.draw_geometries([pcd_load])

Visualize the point cloud.


In [109]:
df_in_radius

,x,y,z
4816,-128.431970,113.026322,75.369524
4820,-129.104083,113.652468,74.412418
4823,-130.046780,112.885092,74.162343
4827,-130.096329,113.919510,73.414898
4830,-131.643768,114.012249,72.020090
...,...,...,...
14021,-130.506977,89.459473,37.500577
14026,-131.816742,88.777172,35.175612
14034,-132.949176,89.279588,32.464307
14044,-133.782664,89.163645,29.790370


In [110]:
np.save("../femur_pcls/minimas/65deg.npy",local_maximum)